<a href="https://colab.research.google.com/github/diegop2110/Talento_Tech_Ciberseguridad/blob/main/Proyecto/passguard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>





## **PassGuard** : Generador y Evaluador de Contraseñas Seguras con Almacenamiento Cifrado.

**Objetivo**: Desarrollar una aplicación de consola en Python que permita a cualquier usuario generar contraseñas fuertes, evaluar la seguridad de las suyas, y almacenarlas de manera cifrada localmente para mantenerlas protegidas contra accesos no autorizados.

por: **Diego Palacios**



# **1- Importación de librerias necesarias**



**os**: provee funciones para interactuar con el sistema operativo (por ejemplo, verificar si existe un archivo, leer variables de entorno, obtener rutas, etc.).

**json**: Permite convertir estructuras de datos de Python (por ejemplo, diccionarios) a cadenas en formato JSON y viceversa (json.dumps, json.loads). Esto es útil para guardar datos en disco de forma legible y estructurada.

**base64**: Proporciona funciones para codificar y decodificar datos en Base64. En este código se emplea para “envolver” una clave derivada en un formato que acepta la librería Fernet (de cryptography).




**secrets**: generar contraseñas seguras aleatorias.

**string:** acceder a letras, números y símbolos.

**getpass**: pedir contraseñas sin mostrarlas en pantalla.

**re:** Proporciona herramientas para trabajar con expresiones regulares, lo cual se utiliza en este código para validar y puntuar la fortaleza de contraseñas (buscando mayúsculas, minúsculas, dígitos, símbolos, etc.).

**cryptography:** importamos el modulo **Fernet** que es una herramienta que permite cifrar y descifrar datos de forma simétrica usando AES internamente, empaquetando la clave en Base64 URL-safe. Con Fernet se garantiza confidencialidad y autenticidad (integridad) de los datos.

**hashes** del paquete **cryptography.hazmat.primitives:**Proporciona algoritmos de hashing (por ejemplo, SHA256) que se usarán en la derivación de la clave (PBKDF2).

**PBKDF2HMAC** del submódulo **kdf.pbkdf2:** es una implementación de la función PBKDF2 (Password-Based Key Derivation Function 2) usando HMAC con un hash (en nuestro caso, SHA256). Se emplea para derivar una clave segura a partir de la contraseña maestra y una sal aleatoria.

**default_backend** desde **cryptography.hazmat.backends:** proporciona la implementación por defecto. (En versiones más recientes, a veces ya no es estrictamente necesario pasarlo, pero el código lo incluye para compatibilidad).





In [1]:
import os
import json
import base64
import secrets
import string
import getpass
import re
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend

# **2- Constantes de seguridad**
Define la constante **VAULT_FILE** con el valor **"vault.dat"**:
Que es el nombre del archivo en disco donde se guardarán los datos cifrados del “vault” (almacén de credenciales)


**SALT_SIZE:** Representa el tamaño (en bytes) de la “sal” que se usará en PBKDF2. Se generan 16 bytes aleatorios para la sal.

**KDF_ITERATIONS**: Indica cuántas iteraciones hará PBKDF2 al derivar la clave. A mayor número, más difícil (y lento) será un ataque por fuerza bruta, pero el cálculo consume más CPU..

**PASSWORD_MIN_LENGTH:**
Se usa para exigir que las contraseñas (tanto la maestra como las generadas) tengan al menos 12 caracteres.




In [2]:
VAULT_FILE = "vault.dat"
SALT_SIZE = 16  # bytes
KDF_ITERATIONS = 390000  # Número de iteraciones para PBKDF2
PASSWORD_MIN_LENGTH = 12

# **3- Funciones criptográficas**

Inicia la definición de la función **derive_key**, que recibe dos parámetros:

**password**: la contraseña maestra en texto plano (tipo str).

**salt**: una secuencia de bytes que actúa como sal (tipo bytes).
Esta función derivará una clave simétrica de 32 bytes (para Fernet) a partir de la contraseña y la sal, usando PBKDF2. Luego devuelve esa clave en formato Base64 URL-safe.

Crea una instancia llamada **kdf** de la clase **PBKDF2HMAC**.

Que se configurará **PBKDF2** (con **HMAC-SHA256**) para luego derivar la clave.
**algorithm=hashes.SHA256()** : Indica que se usará SHA256 como algoritmo de hashing dentro de PBKDF2.

**length=32** : Pide que la clave derivada tenga 32 bytes de longitud, porque Fernet requiere exactamente 32 bytes.

**iterations=KDF_ITERATIONS**:
Indica cuántas iteraciones de HMAC se aplicarán (390 000 en este caso, según la constante)

**backend=default_backend()**: Especifica el backend criptográfico por defecto (necesario en versiones antiguas de cryptography)




In [9]:
def derive_key(password: str, salt: bytes) -> bytes:

    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,  # 32 bytes para Fernet
        salt=salt,
        iterations=KDF_ITERATIONS,
        backend=default_backend()
    )
    return base64.urlsafe_b64encode(kdf.derive(password.encode()))


Convierte la contraseña **password** (cadena) a bytes usando **.encode()**.

Llama a **kdf.derive(...)** para obtener los 32 bytes derivados.

Envuelve esos 32 bytes en **Base64 URL-safe** con **base64.urlsafe_b64encode(...)**.

Devuelve el resultado (clave codificada en Base64) como bytes.

Para qué sirve: Esta es la clave que luego usará Fernet para cifrar/descifrar datos.

In [10]:

def encrypt_data(data: dict, key: bytes) -> bytes:

    f = Fernet(key)
    return f.encrypt(json.dumps(data).encode())

def decrypt_data(token: bytes, key: bytes) -> dict:

    f = Fernet(key)
    try:
        decrypted_data = f.decrypt(token)
        return json.loads(decrypted_data.decode())
    except Exception as e:

        print(f"Error al descifrar: {e}")
        return None

**def encrypt_data(data: dict, key: bytes) -> bytes:** Define la función encrypt_data, que recibe un diccionario (data) y una clave (key) en formato bytes. Devolverá un bloque cifrado (bytes).

**f = Fernet(key)**:Crea un objeto Fernet usando la clave derivada (key). Con f se podrán cifrar y descifrar datos.

**return f.encrypt(json.dumps(data).encode())**: Convierte el diccionario data a una cadena JSON con json.dumps(data).

-Codifica esa cadena JSON a bytes (.encode()).

-Llama a f.encrypt(...) para cifrar los bytes.

-Devuelve los bytes cifrados (que incluyen un encabezado con timestamp y MAC).

**def decrypt_data(token: bytes, key: bytes) -> dict:** Define la función decrypt_data, que toma un bloque cifrado (token) y la misma clave (key) usada para cifrar. Devolverá un diccionario con los datos descifrados (o None en caso de error).

**try:** Inicia un bloque try/except para capturar posibles excepciones (por ejemplo, si la clave es incorrecta o los datos están corruptos).

**decrypted_data = f.decrypt(token)** : Llama a **f.decrypt(...)** para descifrar **token**.
Si el bloque cifrado o la clave no son válidos, esto lanzará una excepción **InvalidToken**.

**except Exception as e:** Captura cualquier excepción que haya ocurrido en el bloque anterior (cifra/desencripta). La excepción se almacena en la variable **e**.

# **4- Gestión del archivo Vault**

**def load_vault(master_password: str) -> (dict, bytes):** Define la función **load_vault**, que recibe la contraseña maestra en texto **(master_password)**. Devuelve una tupla **(datos_descifrados, salt)** donde **datos_descifrados** es un diccionario con las credenciales y **salt** es la sal usada.

**if not os.path.exists(VAULT_FILE):** Comprueba si el archivo **vault.dat** (constante **VAULT_FILE**) NO existe en disco.

**return {}, None** : Si no hay ningún archivo en disco, devuelve un diccionario vacío ({}) y None en lugar de la sal. Esto indica que es la primera vez o que no hay vault creado.

**try:** Inicia un bloque **try/except** para las operaciones de lectura y descifrado del archivo.

**save_vault()** lo vuelve a cifrar y guarda.

**with open(VAULT_FILE, "rb") as f:**
Abre el archivo **vault.dat** en modo lectura binaria ("rb"). Lo asocia a la variable **f**.

**salt = f.read(SALT_SIZE)**: Lee los primeros SALT_SIZE bytes del archivo (16 bytes) para obtener la sal.

**encrypted_data_blob = f.read()**: Lee el resto del archivo (todo después de la sal) y lo guarda en **encrypted_data_blob**. Este bloque es el ciphertext de los datos **JSON** cifrados con Fernet.

**if not encrypted_data_blob:** Verifica si **encrypted_data_blob** está vacío (longitud cero). Eso implica que el archivo solo contenía la sal, sin datos cifrados, lo cual sucedería justo después de inicializar el vault sin almacenar nada aún.

**return {}, salt**: Si no hay datos cifrados, devuelve un diccionario vacío y la **salt** leída.

**derived_key = derive_key(master_password, salt)** :Llama a la función **derive_key** (definida antes) pasando la contraseña maestra y la sal, para obtener la clave **derived_key** (en Base64) que se usará con Fernet.

**data = decrypt_data(encrypted_data_blob, derived_key)**:
Llama a **decrypt_data**, pasándole el bloque cifrado **(encrypted_data_blob)** y la clave derivada **(derived_key)**.
Si la contraseña y la sal coinciden con los que se usaron al cifrar, **data** será el diccionario original de credenciales; de lo contrario, **data** será **None**.

**if data is None:** Verifica si **decrypt_data** devolvió None, lo que indica error (contraseña maestra incorrecta o archivo corrupto).

**return None, None** : Devuelve **(None, None)** para indicar que no fue posible cargar el vault.

**return data, salt** : Si todo salió bien, retorna el diccionario descifrado **(data)** y la sal.

**except Exception as e:** Captura cualquier excepción ocurrida durante la lectura del archivo o el proceso de descifrado.



In [11]:
def load_vault(master_password: str) -> (dict, bytes):
    """Carga el almacén cifrado. Devuelve los datos descifrados y la sal."""
    if not os.path.exists(VAULT_FILE):
        return {}, None  # Almacén vacío si el archivo no existe

    try:
        with open(VAULT_FILE, "rb") as f:
            salt = f.read(SALT_SIZE)
            encrypted_data_blob = f.read()

        if not encrypted_data_blob:
             return {}, salt

        derived_key = derive_key(master_password, salt)
        data = decrypt_data(encrypted_data_blob, derived_key)

        if data is None:
            print("Error: Contraseña maestra incorrecta o almacén corrupto.")
            return None, None
        return data, salt
    except Exception as e:
        print(f"Error al cargar el almacén: {e}")
        return None, None


**def save_vault(data: dict, master_password: str, salt: bytes = None) -> bool:**
Define la función **save_vault**, que recibe:

data: un diccionario con las credenciales a guardar.

**master_password:** la contraseña maestra para derivar la clave.

**salt:** opcional; si es **None**, generará una nueva sal (caso de inicializar o cambiar contraseña maestra).

Devuelve: **True** si pudo guardar correctamente, o **False** si ocurrió algún error.

**try:** Inicia bloque **try/except** para capturar errores durante el cifrado y escritura en disco.

**if salt is None:** Verifica si **salt** fue pasado como **None**. Ese caso ocurre cuando se está creando el vault por primera vez o se está cambiando la contraseña maestra (necesita nueva sal)

**salt = os.urandom(SALT_SIZE)**:  Genera salt aleatoria de **SALT_SIZE** bytes vía **os.urandom**.

**derived_key = derive_key(master_password, salt)**:  Llama a **derive_key** para obtener la clave en Base64 a partir de la contraseña maestra y la sal (nueva o pasada).

**encrypted_data_blob = encrypt_data(data, derived_key)**: Llama a **encrypt_data**, pasándole el diccionario **data** y la clave derivada, retornando el bloque cifrado en bytes.

**with open(VAULT_FILE, "wb") as f:**Abre (o crea) el archivo **vault.dat** en modo escritura binaria **("wb")**, truncando su contenido previo.

**def initialize_vault() -> (str, bytes):** Define la función **initialize_vault**, que no recibe parámetros y devuelve una tupla **(master_password, salt)** si la inicialización fue exitosa, o **(None, None)** si hubo error.

 **while True:** Inicia un bucle infinito para pedir la contraseña maestra hasta que el usuario ingrese dos veces la misma y cumpla requisitos de fortaleza.

**mp = getpass.getpass**: Pide al usuario, de forma oculta en pantalla, la nueva contraseña maestra.

**if mp == mp_confirm:** Comprueba que ambas cadenas ingresadas coincidan.

**if evaluate_strength(mp)[0] in ["Fuerte", "Muy Fuerte"]:**  Llama a **evaluate_strength(mp)**, que devuelve una tupla (fortaleza, sugerencias). Aquí se toma el índice [0] (la categoría de fortaleza) y se comprueba si es “Fuerte” o “Muy Fuerte”.


In [12]:
def save_vault(data: dict, master_password: str, salt: bytes = None) -> bool:
    """Guarda el almacén cifrado."""
    try:
        if salt is None: # Primera vez que se guarda o cambio de contraseña maestra
            salt = os.urandom(SALT_SIZE)

        derived_key = derive_key(master_password, salt)
        encrypted_data_blob = encrypt_data(data, derived_key)

        with open(VAULT_FILE, "wb") as f:
            f.write(salt)
            f.write(encrypted_data_blob)
        return True
    except Exception as e:
        print(f"Error al guardar el almacén: {e}")
        return False

def initialize_vault() -> (str, bytes):
    """Inicializa un nuevo almacén pidiendo una contraseña maestra."""
    print("Parece que es la primera vez que usa PassGuard Pro o el almacén no existe.")
    while True:
        mp = getpass.getpass("Cree una contraseña maestra robusta: ")
        mp_confirm = getpass.getpass("Confirme la contraseña maestra: ")
        if mp == mp_confirm:
            if evaluate_strength(mp)[0] in ["Fuerte", "Muy Fuerte"]: # Evaluar fortaleza de la maestra
                new_salt = os.urandom(SALT_SIZE)
                if save_vault({}, mp, new_salt): # Guardar un almacén vacío inicial
                    print("¡Almacén inicializado exitosamente!")
                    return mp, new_salt
                else:
                    print("Error al inicializar el almacén.")
                    return None, None
            else:
                print("La contraseña maestra es demasiado débil. Intente una más compleja.")
        else:
            print("Las contraseñas maestras no coinciden. Intente de nuevo.")


# **5- Generador y evaluador de contraseñas**

**def generate_password(length: int, use_uppercase: bool, use_lowercase: bool, use_digits: bool, use_symbols: bool) -> str:**

 Define **generate_password**, que recibe:

*  **length**: longitud deseada de la contraseña (entero).

*  **use_uppercase**: si incluir mayúsculas (bool).

* **use_lowercase**: si incluir minúsculas (bool).

*  **use_digits**: si incluir dígitos numéricos (bool).

*  **use_symbols**: si incluir símbolos (bool).
Devuelve una contraseña en texto (str).

**def evaluate_strength(password: str) -> (str, list):** Define la función **evaluate_strength**, que recibe una contraseña en texto (**password**) y devuelve una tupla (fortaleza, sugerencias), donde:

fortaleza es una cadena (p. ej. “Muy Débil”, “Fuerte”).

sugerencias es una lista de cadenas con consejos para mejorar.


In [34]:
def generate_password(length: int, use_uppercase: bool, use_lowercase: bool, use_digits: bool, use_symbols: bool) -> str:
    """Genera una contraseña aleatoria segura."""
    character_pool = ""
    if use_uppercase:
        character_pool += string.ascii_uppercase
    if use_lowercase:
        character_pool += string.ascii_lowercase
    if use_digits:
        character_pool += string.digits
    if use_symbols:
        character_pool += string.punctuation  # Considerar una lista más controlada de símbolos

    if not character_pool:
        return "Error: Debe seleccionar al menos un tipo de carácter."

    password = ''.join(secrets.choice(character_pool) for _ in range(length))
    return password

def evaluate_strength(password: str) -> (str, list):
    """Evalúa la fortaleza de una contraseña."""
    suggestions = []
    score = 0
    if len(password) >= PASSWORD_MIN_LENGTH:
        score += 2
    elif len(password) >= 8:
        score += 1
    else:
        suggestions.append(f"Aumentar la longitud a al menos {PASSWORD_MIN_LENGTH} caracteres.")
    if re.search(r"[A-Z]", password): score += 1
    else: suggestions.append("Incluir letras mayúsculas.")
    if re.search(r"[a-z]", password): score += 1
    else: suggestions.append("Incluir letras minúsculas.")
    if re.search(r"[0-9]", password): score += 1
    else: suggestions.append("Incluir dígitos.")
    if re.search(r"[\W_]", password): score += 1 # \W es no alfanumérico (símbolos)
    else: suggestions.append("Incluir símbolos.")
    if re.search(r"[A-Z]", password): score += 1
    else: suggestions.append("Incluir letras mayúsculas.")
    if re.search(r"[a-z]", password): score += 1
    else: suggestions.append("Incluir letras minúsculas.")
    if re.search(r"[0-9]", password): score += 1
    else: suggestions.append("Incluir dígitos.")
    if re.search(r"[\W_]", password): score += 1 # \W es no alfanumérico (símbolos)
    else: suggestions.append("Incluir símbolos.")
    if score <= 2: strength = "Muy Débil"
    elif score <= 3: strength = "Débil"
    elif score <= 4: strength = "Aceptable"
    elif score <= 5: strength = "Fuerte"
    else: strength = "Muy Fuerte" # score 6
    if password.lower() in ["password", "123456", "qwerty"]:
        strength = "Muy Débil"
        suggestions.append("Evitar contraseñas comunes o secuencias.")

    return strength, suggestions

**Longitud**

*  Mide la longitud de la contraseña (len(password)).

*  Si es mayor o igual que PASSWORD_MIN_LENGTH (12), suma 2 puntos al score.

*  Si está entre 8 y 11 caracteres, suma 1 punto.

*  Si es menor que 8, no suma puntos y añade a suggestions el mensaje para alargarla a al menos 12.

**Variedad de caracteres**

* Usa re.search(r"[A-Z]", password) para verificar si hay al menos una letra mayúscula.

* Si la encuentra, suma 1 punto.

* Si no la encuentra, agrega a suggestions la recomendación “Incluir letras mayúsculas.”

**Clasificación (simple)**

Según la suma total de puntos (score), asigna la variable strength a una de estas categorías:

* 0–2 → “Muy Débil”

* 3 → “Débil”

* 4 → “Aceptable”

* 5 → “Fuerte”

* 6 o más → “Muy Fuerte”



**Detección de patrones muy básicos**

Convierte password a minúsculas y verifica si coincide exactamente con “password”, “123456” o “qwerty”. Si es alguna de ellas, anula la categoría y la pone en “Muy Débil” y añade la sugerencia de evitar patrones básicos


# **6- Funciones del menú**

**handle_generate_password()**: Define la función handle_generate_password, que recibe:

vault_data: el diccionario actual con las credenciales (cargado desde el vault).

master_password: la contraseña maestra en uso durante esta sesión.

salt: la sal usada para cifrar/descifrar.

Para qué sirve: Muestra un submenú interactivo para que el usuario genere una nueva contraseña, la evalúe y, opcionalmente, la guarde en el vault.

**print("\n--- Generar Nueva Contraseña ---")**: Imprime en pantalla el encabezado del submenú. El \n agrega una línea en blanco antes para que se vea más claro.


In [35]:
def handle_generate_password(vault_data: dict, master_password: str, salt: bytes):

    print("\n--- Generar Nueva Contraseña ---")

**try:**  Inicia un bloque de tipo try/except para capturar errores al leer la longitud.

**length = int(input(f"Longitud de la contraseña (mínimo {PASSWORD_MIN_LENGTH}): "))** : Muestra un mensaje solicitando al usuario la longitud deseada (como entero). Si el usuario ingresa algo que no se puede convertir a entero, saltará a except.

In [ ]:

    try:
        length = int(input(f"Longitud de la contraseña (mínimo {PASSWORD_MIN_LENGTH}): "))
        if length < PASSWORD_MIN_LENGTH : length = PASSWORD_MIN_LENGTH
    except ValueError:
        print("Longitud inválida. Usando por defecto 16.")
        length = 16

    use_upper = input("¿Usar mayúsculas? (s/N): ").lower() == 's'
    use_lower = input("¿Usar minúsculas? (S/n): ").lower() != 'n' # Por defecto sí
    use_digits = input("¿Usar dígitos? (S/n): ").lower() != 'n'    # Por defecto sí
    use_symbols = input("¿Usar símbolos? (S/n): ").lower() != 'n' # Por defecto sí

    if not (use_upper or use_lower or use_digits or use_symbols):
        print("Debe seleccionar al menos un conjunto de caracteres. Usando minúsculas por defecto.")
        use_lower = True

    new_password = generate_password(length, use_upper, use_lower, use_digits, use_symbols)
    print(f"\nContraseña generada: {new_password}")
    strength, suggestions = evaluate_strength(new_password)
    print(f"Fortaleza: {strength}")

    if input("¿Guardar esta contraseña en el almacén? (s/N): ").lower() == 's':
        label = input("Etiqueta para esta contraseña (ej. 'correo', 'banco'): ")
        username = input(f"Nombre de usuario para '{label}' (opcional): ")
        if label in vault_data:
            if input(f"Ya existe una entrada para '{label}'. ¿Sobrescribir? (s/N): ").lower() != 's':
                return
        vault_data[label] = {"username": username, "password": new_password}
        if save_vault(vault_data, master_password, salt):
            print(f"Contraseña para '{label}' guardada exitosamente.")
        else:
            print("Error al guardar la contraseña.")
    print("\nADVERTENCIA: Si copió la contraseña, bórrela del portapapeles manualmente por seguridad.")

**def handle_evaluate_password():**

Pide al usuario una contraseña (oculta).

Usa **evaluate_strength()** para puntuarla.

Muestra categoría y posibles mejoras (incluir mayúsculas, símbolos, etc.).

In [ ]:
def handle_evaluate_password():
    print("\n--- Evaluar Fortaleza de Contraseña ---")
    password_to_eval = getpass.getpass("Ingrese la contraseña a evaluar: ")
    strength, suggestions = evaluate_strength(password_to_eval)
    print(f"\nFortaleza: {strength}")
    if suggestions:
        print("Sugerencias para mejorar:")
        for sug in suggestions:
            print(f"- {sug}")

**(handle_add_credential):**

Solicita etiqueta, usuario y contraseña directamente.

Evalúa la fortaleza; si es “Débil” o “Muy Débil”, pregunta si quiere continuar.

Inserta o sobrescribe la entrada en **vault_data** y llama a **save_vault()**.

In [ ]:
def handle_add_credential(vault_data: dict, master_password: str, salt: bytes):
    print("\n--- Guardar/Actualizar Credencial ---")
    label = input("Etiqueta para la credencial (ej. 'web_servicio_X'): ").strip()
    if not label:
        print("La etiqueta no puede estar vacía.")
        return

    username = input(f"Nombre de usuario para '{label}': ")
    password = getpass.getpass(f"Contraseña para '{label}': ")

    strength, _ = evaluate_strength(password)
    print(f"Fortaleza de la contraseña ingresada: {strength}")
    if strength in ["Muy Débil", "Débil"]:
        if input("La contraseña es débil. ¿Continuar guardando? (s/N): ").lower() != 's':
            return

    if label in vault_data and \
       input(f"Ya existe una entrada para '{label}'. ¿Sobrescribir? (s/N): ").lower() != 's':
        return

    vault_data[label] = {"username": username, "password": password}
    if save_vault(vault_data, master_password, salt):
        print(f"Credencial para '{label}' guardada/actualizada exitosamente.")
    else:
        print("Error al guardar la credencial.")

**(handle_view_credentials)**:

Opcionalmente filtra las etiquetas por texto ingresado.

Lista usuario y una máscara de asteriscos.

Si el usuario lo pide, revela la contraseña en texto claro.

In [ ]:
def handle_view_credentials(vault_data: dict):
    print("\n--- Ver Credenciales Guardadas ---")
    if not vault_data:
        print("El almacén está vacío.")
        return

    search_term = input("Buscar por etiqueta (dejar en blanco para ver todas): ").strip().lower()
    found = False
    for label, creds in vault_data.items():
        if search_term in label.lower() or not search_term:
            found = True
            print(f"\nSitio/Servicio: {label}")
            print(f"  Usuario: {creds.get('username', 'N/A')}")
            if input("  ¿Mostrar contraseña? (s/N): ").lower() == 's':
                print(f"  Contraseña: {creds['password']}")
                print("  ADVERTENCIA: Borre la contraseña de la pantalla y del portapapeles si la copió.")
            else:
                print(f"  Contraseña: {'*' * len(creds['password'])}")
    if not found and search_term:
        print(f"No se encontraron credenciales que coincidan con '{search_term}'.")

**(handle_delete_credential)**:

Pide la etiqueta a borrar.

Si existe y confirma, la elimina del diccionario y vuelve a guardar el vault.

Si no existe, informa al usuario.

In [ ]:
def handle_delete_credential(vault_data: dict, master_password: str, salt: bytes):
    print("\n--- Eliminar Credencial ---")
    if not vault_data:
        print("El almacén está vacío. Nada que eliminar.")
        return

    label_to_delete = input("Etiqueta de la credencial a eliminar: ").strip()
    if label_to_delete in vault_data:
        if input(f"¿Está seguro de que desea eliminar la credencial para '{label_to_delete}'? Esta acción es irreversible. (s/N): ").lower() == 's':
            del vault_data[label_to_delete]
            if save_vault(vault_data, master_password, salt):
                print(f"Credencial para '{label_to_delete}' eliminada exitosamente.")
            else:
                print("Error al guardar los cambios tras eliminar la credencial.")
        else:
            print("Eliminación cancelada.")
    else:
        print(f"No se encontró ninguna credencial con la etiqueta '{label_to_delete}'.")

**(handle_change_master_password)** :

Verifica la contraseña actual para evitar accesos no autorizados.

Solicita la nueva dos veces y comprueba coincidencia.

Evalúa su fortaleza.

Genera una sal nueva y vuelve a cifrar todo el vault con la nueva clave.

In [ ]:
def handle_change_master_password(current_vault_data: dict, current_master_password: str, current_salt: bytes) -> (str, bytes, bool):
    print("\n--- Cambiar Contraseña Maestra ---")
    print("ADVERTENCIA: Esta es una operación crítica. Si olvida la nueva contraseña, perderá acceso a sus datos.")

    # Validar contraseña maestra actual (aunque ya deberíamos tenerla)
    # Esto es más una re-confirmación antes de un cambio tan grande.
    print("Por seguridad, ingrese su contraseña maestra ACTUAL:")
    mp_current_check = getpass.getpass("Contraseña maestra actual: ")
    if mp_current_check != current_master_password: # Podríamos re-derivar la clave para estar 100% seguros
        print("La contraseña maestra actual no es correcta. Cambio cancelado.")
        return current_master_password, current_salt, False # No se cambió

    while True:
        new_mp = getpass.getpass("Ingrese la NUEVA contraseña maestra: ")
        new_mp_confirm = getpass.getpass("Confirme la NUEVA contraseña maestra: ")

        if new_mp == new_mp_confirm:
            strength, _ = evaluate_strength(new_mp)
            if strength not in ["Fuerte", "Muy Fuerte"]:
                print(f"La nueva contraseña maestra es {strength}. Por favor, elija una más robusta.")
                if input("¿Intentar de nuevo? (S/n): ").lower() == 'n':
                    print("Cambio de contraseña maestra cancelado.")
                    return current_master_password, current_salt, False
                continue # Vuelve a pedir la nueva contraseña

            # Generar una NUEVA SAL para la nueva contraseña maestra
            new_salt = os.urandom(SALT_SIZE)
            if save_vault(current_vault_data, new_mp, new_salt): # Guardar con nueva clave y nueva sal
                print("¡Contraseña maestra cambiada exitosamente!")
                print("Asegúrese de recordar su nueva contraseña maestra.")
                return new_mp, new_salt, True # Se cambió
            else:
                print("Error crítico al guardar el almacén con la nueva contraseña maestra.")
                print("Se intentará restaurar con la contraseña anterior. NO CIERRE LA APLICACIÓN.")
                if save_vault(current_vault_data, current_master_password, current_salt):
                     print("Restauración con contraseña anterior exitosa. El cambio de contraseña falló.")
                else:
                     print("FALLO CRÍTICO: No se pudo restaurar el almacén. Datos podrían estar en riesgo.")
                return current_master_password, current_salt, False
        else:
            print("Las nuevas contraseñas maestras no coinciden. Intente de nuevo.")
            if input("¿Intentar de nuevo? (S/n): ").lower() == 'n':
                print("Cambio de contraseña maestra cancelado.")
                return current_master_password, current_salt, False


Codigo completo funcionando

In [2]:
import os
import json
import base64
import secrets
import string
import getpass
import re
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend

# --- Constantes ---
VAULT_FILE = "vault.dat"
SALT_SIZE = 16  # bytes
KDF_ITERATIONS = 390000  # Número de iteraciones para PBKDF2 (ajustar según necesidad/rendimiento)
PASSWORD_MIN_LENGTH = 12

# --- Funciones de Criptografía ---

def derive_key(password: str, salt: bytes) -> bytes:
    """Deriva una clave de cifrado a partir de la contraseña maestra y la sal usando PBKDF2."""
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,  # 32 bytes para Fernet
        salt=salt,
        iterations=KDF_ITERATIONS,
        backend=default_backend()
    )
    return base64.urlsafe_b64encode(kdf.derive(password.encode()))

def encrypt_data(data: dict, key: bytes) -> bytes:
    """Cifra los datos (diccionario) usando Fernet."""
    f = Fernet(key)
    return f.encrypt(json.dumps(data).encode())

def decrypt_data(token: bytes, key: bytes) -> dict:
    """Descifra los datos usando Fernet y los devuelve como diccionario."""
    f = Fernet(key)
    try:
        decrypted_data = f.decrypt(token)
        return json.loads(decrypted_data.decode())
    except Exception as e:
        # Podría ser InvalidToken si la clave es incorrecta o los datos están corruptos
        print(f"Error al descifrar: {e}")
        return None

# --- Funciones del Almacén (Vault) ---

def load_vault(master_password: str) -> (dict, bytes):
    """Carga el almacén cifrado. Devuelve los datos descifrados y la sal."""
    if not os.path.exists(VAULT_FILE):
        return {}, None  # Almacén vacío si el archivo no existe

    try:
        with open(VAULT_FILE, "rb") as f:
            salt = f.read(SALT_SIZE)
            encrypted_data_blob = f.read()

        if not encrypted_data_blob: # Si el archivo solo tiene la sal (almacén recién creado y vacío)
             return {}, salt

        derived_key = derive_key(master_password, salt)
        data = decrypt_data(encrypted_data_blob, derived_key)

        if data is None:
            print("Error: Contraseña maestra incorrecta o almacén corrupto.")
            return None, None
        return data, salt
    except Exception as e:
        print(f"Error al cargar el almacén: {e}")
        return None, None

def save_vault(data: dict, master_password: str, salt: bytes = None) -> bool:
    """Guarda el almacén cifrado."""
    try:
        if salt is None: # Primera vez que se guarda o cambio de contraseña maestra
            salt = os.urandom(SALT_SIZE)

        derived_key = derive_key(master_password, salt)
        encrypted_data_blob = encrypt_data(data, derived_key)

        with open(VAULT_FILE, "wb") as f:
            f.write(salt)
            f.write(encrypted_data_blob)
        return True
    except Exception as e:
        print(f"Error al guardar el almacén: {e}")
        return False

def initialize_vault() -> (str, bytes):
    """Inicializa un nuevo almacén pidiendo una contraseña maestra."""
    print("Parece que es la primera vez que usa PassGuard Pro o el almacén no existe.")
    while True:
        mp = getpass.getpass("Cree una contraseña maestra robusta: ")
        mp_confirm = getpass.getpass("Confirme la contraseña maestra: ")
        if mp == mp_confirm:
            if evaluate_strength(mp)[0] in ["Fuerte", "Muy Fuerte"]: # Evaluar fortaleza de la maestra
                new_salt = os.urandom(SALT_SIZE)
                if save_vault({}, mp, new_salt): # Guardar un almacén vacío inicial
                    print("¡Almacén inicializado exitosamente!")
                    return mp, new_salt
                else:
                    print("Error al inicializar el almacén.")
                    return None, None
            else:
                print("La contraseña maestra es demasiado débil. Intente una más compleja.")
        else:
            print("Las contraseñas maestras no coinciden. Intente de nuevo.")


# --- Funciones de Generación y Evaluación de Contraseñas ---

def generate_password(length: int, use_uppercase: bool, use_lowercase: bool, use_digits: bool, use_symbols: bool) -> str:
    """Genera una contraseña aleatoria segura."""
    character_pool = ""
    if use_uppercase:
        character_pool += string.ascii_uppercase
    if use_lowercase:
        character_pool += string.ascii_lowercase
    if use_digits:
        character_pool += string.digits
    if use_symbols:
        character_pool += string.punctuation  # Considerar una lista más controlada de símbolos

    if not character_pool:
        return "Error: Debe seleccionar al menos un tipo de carácter."

    password = ''.join(secrets.choice(character_pool) for _ in range(length))
    return password

def evaluate_strength(password: str) -> (str, list):
    """Evalúa la fortaleza de una contraseña."""
    suggestions = []
    score = 0

    # Longitud
    if len(password) >= PASSWORD_MIN_LENGTH:
        score += 2
    elif len(password) >= 8:
        score += 1
    else:
        suggestions.append(f"Aumentar la longitud a al menos {PASSWORD_MIN_LENGTH} caracteres.")

    # Variedad de caracteres
    if re.search(r"[A-Z]", password): score += 1
    else: suggestions.append("Incluir letras mayúsculas.")
    if re.search(r"[a-z]", password): score += 1
    else: suggestions.append("Incluir letras minúsculas.")
    if re.search(r"[0-9]", password): score += 1
    else: suggestions.append("Incluir dígitos.")
    if re.search(r"[\W_]", password): score += 1 # \W es no alfanumérico (símbolos)
    else: suggestions.append("Incluir símbolos.")

    # Clasificación (simple)
    if score <= 2: strength = "Muy Débil"
    elif score <= 3: strength = "Débil"
    elif score <= 4: strength = "Aceptable"
    elif score <= 5: strength = "Fuerte"
    else: strength = "Muy Fuerte" # score 6

    # Detección de patrones muy básicos (opcional, expandir)
    if password.lower() in ["password", "123456", "qwerty"]:
        strength = "Muy Débil"
        suggestions.append("Evitar contraseñas comunes o secuencias.")

    return strength, suggestions

# --- Funciones del Menú ---

def handle_generate_password(vault_data: dict, master_password: str, salt: bytes):
    print("\n--- Generar Nueva Contraseña ---")
    try:
        length = int(input(f"Longitud de la contraseña (mínimo {PASSWORD_MIN_LENGTH}): "))
        if length < PASSWORD_MIN_LENGTH : length = PASSWORD_MIN_LENGTH
    except ValueError:
        print("Longitud inválida. Usando por defecto 16.")
        length = 16

    use_upper = input("¿Usar mayúsculas? (s/N): ").lower() == 's'
    use_lower = input("¿Usar minúsculas? (S/n): ").lower() != 'n' # Por defecto sí
    use_digits = input("¿Usar dígitos? (S/n): ").lower() != 'n'    # Por defecto sí
    use_symbols = input("¿Usar símbolos? (S/n): ").lower() != 'n' # Por defecto sí

    if not (use_upper or use_lower or use_digits or use_symbols):
        print("Debe seleccionar al menos un conjunto de caracteres. Usando minúsculas por defecto.")
        use_lower = True

    new_password = generate_password(length, use_upper, use_lower, use_digits, use_symbols)
    print(f"\nContraseña generada: {new_password}")
    strength, suggestions = evaluate_strength(new_password)
    print(f"Fortaleza: {strength}")

    if input("¿Guardar esta contraseña en el almacén? (s/N): ").lower() == 's':
        label = input("Etiqueta para esta contraseña (ej. 'correo', 'banco'): ")
        username = input(f"Nombre de usuario para '{label}' (opcional): ")
        if label in vault_data:
            if input(f"Ya existe una entrada para '{label}'. ¿Sobrescribir? (s/N): ").lower() != 's':
                return
        vault_data[label] = {"username": username, "password": new_password}
        if save_vault(vault_data, master_password, salt):
            print(f"Contraseña para '{label}' guardada exitosamente.")
        else:
            print("Error al guardar la contraseña.")
    print("\nADVERTENCIA: Si copió la contraseña, bórrela del portapapeles manualmente por seguridad.")


def handle_evaluate_password():
    print("\n--- Evaluar Fortaleza de Contraseña ---")
    password_to_eval = getpass.getpass("Ingrese la contraseña a evaluar: ")
    strength, suggestions = evaluate_strength(password_to_eval)
    print(f"\nFortaleza: {strength}")
    if suggestions:
        print("Sugerencias para mejorar:")
        for sug in suggestions:
            print(f"- {sug}")

def handle_add_credential(vault_data: dict, master_password: str, salt: bytes):
    print("\n--- Guardar/Actualizar Credencial ---")
    label = input("Etiqueta para la credencial (ej. 'web_servicio_X'): ").strip()
    if not label:
        print("La etiqueta no puede estar vacía.")
        return

    username = input(f"Nombre de usuario para '{label}': ")
    password = getpass.getpass(f"Contraseña para '{label}': ")

    strength, _ = evaluate_strength(password)
    print(f"Fortaleza de la contraseña ingresada: {strength}")
    if strength in ["Muy Débil", "Débil"]:
        if input("La contraseña es débil. ¿Continuar guardando? (s/N): ").lower() != 's':
            return

    if label in vault_data and \
       input(f"Ya existe una entrada para '{label}'. ¿Sobrescribir? (s/N): ").lower() != 's':
        return

    vault_data[label] = {"username": username, "password": password}
    if save_vault(vault_data, master_password, salt):
        print(f"Credencial para '{label}' guardada/actualizada exitosamente.")
    else:
        print("Error al guardar la credencial.")


def handle_view_credentials(vault_data: dict):
    print("\n--- Ver Credenciales Guardadas ---")
    if not vault_data:
        print("El almacén está vacío.")
        return

    search_term = input("Buscar por etiqueta (dejar en blanco para ver todas): ").strip().lower()
    found = False
    for label, creds in vault_data.items():
        if search_term in label.lower() or not search_term:
            found = True
            print(f"\nSitio/Servicio: {label}")
            print(f"  Usuario: {creds.get('username', 'N/A')}")
            if input("  ¿Mostrar contraseña? (s/N): ").lower() == 's':
                print(f"  Contraseña: {creds['password']}")
                print("  ADVERTENCIA: Borre la contraseña de la pantalla y del portapapeles si la copió.")
            else:
                print(f"  Contraseña: {'*' * len(creds['password'])}")
    if not found and search_term:
        print(f"No se encontraron credenciales que coincidan con '{search_term}'.")

def handle_delete_credential(vault_data: dict, master_password: str, salt: bytes):
    print("\n--- Eliminar Credencial ---")
    if not vault_data:
        print("El almacén está vacío. Nada que eliminar.")
        return

    label_to_delete = input("Etiqueta de la credencial a eliminar: ").strip()
    if label_to_delete in vault_data:
        if input(f"¿Está seguro de que desea eliminar la credencial para '{label_to_delete}'? Esta acción es irreversible. (s/N): ").lower() == 's':
            del vault_data[label_to_delete]
            if save_vault(vault_data, master_password, salt):
                print(f"Credencial para '{label_to_delete}' eliminada exitosamente.")
            else:
                print("Error al guardar los cambios tras eliminar la credencial.")
        else:
            print("Eliminación cancelada.")
    else:
        print(f"No se encontró ninguna credencial con la etiqueta '{label_to_delete}'.")


def handle_change_master_password(current_vault_data: dict, current_master_password: str, current_salt: bytes) -> (str, bytes, bool):
    print("\n--- Cambiar Contraseña Maestra ---")
    print("ADVERTENCIA: Esta es una operación crítica. Si olvida la nueva contraseña, perderá acceso a sus datos.")

    # Validar contraseña maestra actual (aunque ya deberíamos tenerla)
    # Esto es más una re-confirmación antes de un cambio tan grande.
    print("Por seguridad, ingrese su contraseña maestra ACTUAL:")
    mp_current_check = getpass.getpass("Contraseña maestra actual: ")
    if mp_current_check != current_master_password: # Podríamos re-derivar la clave para estar 100% seguros
        print("La contraseña maestra actual no es correcta. Cambio cancelado.")
        return current_master_password, current_salt, False # No se cambió

    while True:
        new_mp = getpass.getpass("Ingrese la NUEVA contraseña maestra: ")
        new_mp_confirm = getpass.getpass("Confirme la NUEVA contraseña maestra: ")

        if new_mp == new_mp_confirm:
            strength, _ = evaluate_strength(new_mp)
            if strength not in ["Fuerte", "Muy Fuerte"]:
                print(f"La nueva contraseña maestra es {strength}. Por favor, elija una más robusta.")
                if input("¿Intentar de nuevo? (S/n): ").lower() == 'n':
                    print("Cambio de contraseña maestra cancelado.")
                    return current_master_password, current_salt, False
                continue # Vuelve a pedir la nueva contraseña

            # Generar una NUEVA SAL para la nueva contraseña maestra
            new_salt = os.urandom(SALT_SIZE)
            if save_vault(current_vault_data, new_mp, new_salt): # Guardar con nueva clave y nueva sal
                print("¡Contraseña maestra cambiada exitosamente!")
                print("Asegúrese de recordar su nueva contraseña maestra.")
                return new_mp, new_salt, True # Se cambió
            else:
                print("Error crítico al guardar el almacén con la nueva contraseña maestra.")
                print("Se intentará restaurar con la contraseña anterior. NO CIERRE LA APLICACIÓN.")
                if save_vault(current_vault_data, current_master_password, current_salt):
                     print("Restauración con contraseña anterior exitosa. El cambio de contraseña falló.")
                else:
                     print("FALLO CRÍTICO: No se pudo restaurar el almacén. Datos podrían estar en riesgo.")
                return current_master_password, current_salt, False
        else:
            print("Las nuevas contraseñas maestras no coinciden. Intente de nuevo.")
            if input("¿Intentar de nuevo? (S/n): ").lower() == 'n':
                print("Cambio de contraseña maestra cancelado.")
                return current_master_password, current_salt, False


# --- Bucle Principal ---
def main():
    print("Bienvenido a PassGuard Pro")

    master_password_session = None
    salt_session = None
    vault_data_session = None

    # Intentar cargar o inicializar el almacén
    if os.path.exists(VAULT_FILE):
        while True:
            mp_attempt = getpass.getpass("Ingrese su contraseña maestra para desbloquear el almacén: ")
            vault_data_session, salt_session = load_vault(mp_attempt)
            if vault_data_session is not None: # Carga exitosa
                master_password_session = mp_attempt
                print("Almacén desbloqueado.")
                break
            else:
                # load_vault ya imprime el error (contraseña incorrecta o corrupto)
                if input("¿Intentar de nuevo? (S/n): ").lower() == 'n':
                    print("Saliendo de PassGuard Pro.")
                    return
    else:
        master_password_session, salt_session = initialize_vault()
        if master_password_session is None:
            print("No se pudo inicializar el almacén. Saliendo.")
            return
        vault_data_session, _ = load_vault(master_password_session) # Cargar el almacén recién creado
        if vault_data_session is None:
            print("Error crítico al cargar el almacén recién inicializado. Saliendo.")
            return


    while True:
        print("\n--- Menú Principal ---")
        print("1. Generar nueva contraseña")
        print("2. Evaluar fortaleza de contraseña")
        print("3. Guardar/Actualizar credencial en el almacén")
        print("4. Ver/Buscar credenciales guardadas")
        print("5. Eliminar credencial del almacén")
        print("6. Cambiar Contraseña Maestra")
        print("7. Salir")

        choice = input("Seleccione una opción: ")

        if choice == '1':
            handle_generate_password(vault_data_session, master_password_session, salt_session)
        elif choice == '2':
            handle_evaluate_password()
        elif choice == '3':
            handle_add_credential(vault_data_session, master_password_session, salt_session)
        elif choice == '4':
            handle_view_credentials(vault_data_session)
        elif choice == '5':
            handle_delete_credential(vault_data_session, master_password_session, salt_session)
        elif choice == '6':
            new_mp, new_salt, changed = handle_change_master_password(vault_data_session, master_password_session, salt_session)
            if changed:
                master_password_session = new_mp
                salt_session = new_salt
        elif choice == '7':
            print("Cerrando PassGuard Pro de forma segura...")
            # Considerar borrar variables sensibles de memoria si es posible (difícil en Python)
            break
        else:
            print("Opción no válida. Intente de nuevo.")

    print("¡Gracias por usar PassGuard Pro!")

if __name__ == "__main__":
    main()

Bienvenido a PassGuard Pro
Parece que es la primera vez que usa PassGuard Pro o el almacén no existe.
Cree una contraseña maestra robusta: ··········
Confirme la contraseña maestra: ··········
¡Almacén inicializado exitosamente!

--- Menú Principal ---
1. Generar nueva contraseña
2. Evaluar fortaleza de contraseña
3. Guardar/Actualizar credencial en el almacén
4. Ver/Buscar credenciales guardadas
5. Eliminar credencial del almacén
6. Cambiar Contraseña Maestra
7. Salir
Seleccione una opción: 1

--- Generar Nueva Contraseña ---
Longitud de la contraseña (mínimo 12): 12
¿Usar mayúsculas? (s/N): s
¿Usar minúsculas? (S/n): s
¿Usar dígitos? (S/n): s
¿Usar símbolos? (S/n): s

Contraseña generada: nd%xoJ,5@{5u
Fortaleza: Muy Fuerte
¿Guardar esta contraseña en el almacén? (s/N): s
Etiqueta para esta contraseña (ej. 'correo', 'banco'): Banco
Nombre de usuario para 'Banco' (opcional): Diego
Contraseña para 'Banco' guardada exitosamente.

ADVERTENCIA: Si copió la contraseña, bórrela del portapape

KeyboardInterrupt: Interrupted by user

**Importamos la libreria gradio para intentar tener una mejor interfaz grafica**

In [3]:
import gradio as gr
print(gr.__version__)


5.31.0


In [4]:
!pip install --upgrade "gradio>=3.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 30.9 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.1
    Uninstalling gradio_client-1.10.1:
      Successfully uninstalled gradio_client-1.10.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.31.0
    Uninstalling gradio-5.31.0:
      Successfully uninstalled gradio-5.31.0


In [5]:
import os, json, base64, secrets, string, re, shutil
import pandas as pd
import gradio as gr
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes, kdf
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend

# --- Configuración ---
VAULT_FILE         = "vault.dat"
RECOVERY_FILE      = "vault_recovery.dat"
SALT_SIZE          = 16
KDF_ITERATIONS     = 390000
MIN_PASSWORD_LEN   = 12

session = {
    "password": "",
    "salt":     b"",
    "data":     {},
    "authenticated": False
}

# --- Criptografía básica ---
def derive_key(password: str, salt: bytes) -> bytes:
    k = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=KDF_ITERATIONS,
        backend=default_backend()
    )
    return base64.urlsafe_b64encode(k.derive(password.encode()))

def encrypt_data(data: dict, key: bytes) -> bytes:
    return Fernet(key).encrypt(json.dumps(data).encode())

def decrypt_data(token: bytes, key: bytes) -> dict:
    try:
        return json.loads(Fernet(key).decrypt(token).decode())
    except:
        return None

# --- Vault: guardar, cargar, backup/restore, reset completo ---
def save_vault(data, master_pwd, salt):
    key = derive_key(master_pwd, salt)
    with open(VAULT_FILE, "wb") as f:
        f.write(salt)
        f.write(encrypt_data(data, key))

def load_vault(master_pwd):
    if not os.path.exists(VAULT_FILE):
        s = os.urandom(SALT_SIZE)
        save_vault({}, master_pwd, s)
        session.update(password=master_pwd, salt=s, data={}, authenticated=True)
        return "🆕 Nuevo vault creado."
    with open(VAULT_FILE, "rb") as f:
        s, blob = f.read(SALT_SIZE), f.read()
    key = derive_key(master_pwd, s)
    data = decrypt_data(blob, key)
    if data is None:
        session["authenticated"] = False
        return "❌ Contraseña maestra incorrecta."
    session.update(password=master_pwd, salt=s, data=data, authenticated=True)
    return "🔓 Vault desbloqueado."

def setup_recovery(recovery_pwd):
    if not session["authenticated"]:
        return "❌ Autentícate primero."
    s = os.urandom(SALT_SIZE)
    key = derive_key(recovery_pwd, s)
    with open(RECOVERY_FILE, "wb") as f:
        f.write(s)
        f.write(encrypt_data(session["data"], key))
    return "🔑 Recovery configurado."

def recover_with_recovery(recovery_pwd, new_master_pwd):
    if not os.path.exists(RECOVERY_FILE):
        return "❌ No hay archivo de recovery."
    with open(RECOVERY_FILE, "rb") as f:
        s, blob = f.read(SALT_SIZE), f.read()
    key = derive_key(recovery_pwd, s)
    data = decrypt_data(blob, key)
    if data is None:
        return "❌ Recovery password incorrecto."
    new_salt = os.urandom(SALT_SIZE)
    save_vault(data, new_master_pwd, new_salt)
    session.update(password=new_master_pwd, salt=new_salt, data=data, authenticated=True)
    return "✅ Vault recuperado con nueva contraseña maestra."

def reset_all(master_pwd):
    if not session["authenticated"] or master_pwd != session["password"]:
        return "❌ No autorizado."
    if os.path.exists(VAULT_FILE):
        os.remove(VAULT_FILE)
    session.update(password="", salt=b"", data={}, authenticated=False)
    return "⚠️ Vault y contraseña maestra eliminados. Pérdida total."

def exit_app():
    session["authenticated"] = False
    return "👋 Sesión cerrada."

# --- Funciones de uso diario ---
def generate_password(count, use_upper, use_lower, use_digits, use_symbols):
    length = int(count)
    pool = ""
    if use_upper: pool += string.ascii_uppercase
    if use_lower: pool += string.ascii_lowercase
    if use_digits: pool += string.digits
    if use_symbols: pool += string.punctuation
    if not pool:
        return "❌ Selecciona al menos un tipo de carácter."
    return "".join(secrets.choice(pool) for _ in range(length))

def add_credential(label, usuario, pwd, master_pwd):
    if not session["authenticated"] or master_pwd != session["password"]:
        return "❌ Autenticación fallida."
    session["data"][label] = {"usuario": usuario, "password": pwd}
    save_vault(session["data"], session["password"], session["salt"])
    return f"✅ Credencial '{label}' guardada."

def delete_credential(label, master_pwd):
    if not session["authenticated"] or master_pwd != session["password"]:
        return "❌ Autenticación fallida."
    if label in session["data"]:
        del session["data"][label]
        save_vault(session["data"], session["password"], session["salt"])
        return f"🗑️ Credencial '{label}' eliminada."
    return "⚠️ Etiqueta no encontrada."

def get_credentials_df(master_pwd):
    if not session["authenticated"] or master_pwd != session["password"]:
        return pd.DataFrame(columns=["Etiqueta", "Usuario", "Contraseña"])
    df = pd.DataFrame.from_dict(session["data"], orient="index")
    return df.reset_index().rename(columns={"index":"Etiqueta","usuario":"Usuario","password":"Contraseña"})

def evaluate_strength(password):
    score = sum([
        len(password) >= MIN_PASSWORD_LEN,
        len(password) >= 8,
        bool(re.search(r"[A-Z]", password)),
        bool(re.search(r"[a-z]", password)),
        bool(re.search(r"[0-9]", password)),
        bool(re.search(r"[\\W_]", password))
    ])
    levels = ["Muy Débil","Débil","Aceptable","Fuerte","Muy Fuerte","Excelente"]
    return levels[min(score, len(levels)-1)]

def reset_master_password(old_mp, new_mp):
    if not session["authenticated"] or old_mp != session["password"]:
        return "❌ Validación fallida."
    if evaluate_strength(new_mp) in ["Muy Débil","Débil"]:
        return "⚠️ Usa una contraseña más fuerte."
    new_salt = os.urandom(SALT_SIZE)
    save_vault(session["data"], new_mp, new_salt)
    session.update(password=new_mp, salt=new_salt)
    return "🔑 Contraseña maestra actualizada."

# --- Interfaz Gradio ---
with gr.Blocks() as app:
    gr.Markdown("## 🛡️ PassGuard ")

    with gr.Tab("🔐 Autenticación"):
        master_input = gr.Textbox(label="Contraseña Maestra", type="password")
        auth_status  = gr.Textbox(label="Estado")
        gr.Button("Ingresar / Crear vault").click(load_vault, master_input, auth_status)

    with gr.Tab("🔑 Generar y Guardar"):
        cnt   = gr.Number(value=12, label="Número exacto de caracteres")
        upper = gr.Checkbox(label="Mayúsculas")
        lower = gr.Checkbox(label="Minúsculas", value=True)
        digs  = gr.Checkbox(label="Dígitos", value=True)
        syms  = gr.Checkbox(label="Símbolos")
        gen_out = gr.Textbox(label="Contraseña Generada", interactive=False)
        gr.Button("Generar").click(generate_password, [cnt, upper, lower, digs, syms], gen_out)

        tag      = gr.Textbox(label="Etiqueta")
        usr      = gr.Textbox(label="Usuario (opcional)")
        master2  = gr.Textbox(label="Contraseña Maestra", type="password")
        save_res = gr.Textbox(label="Resultado")
        gr.Button("Guardar credencial").click(add_credential, [tag, usr, gen_out, master2], save_res)
        gr.Button("Eliminar credencial").click(delete_credential, [tag, master2], save_res)

    with gr.Tab("📋 Ver credenciales"):
        master3 = gr.Textbox(label="Contraseña Maestra", type="password")
        cred_df = gr.Dataframe(headers=["Etiqueta","Usuario","Contraseña"], interactive=False)
        gr.Button("Mostrar credenciales").click(get_credentials_df, master3, cred_df)

    with gr.Tab("⚙️ Configurar Recovery"):
        rec_pwd = gr.Textbox(label="Recovery Password", type="password")
        rec_stat= gr.Textbox(label="Estado")
        gr.Button("Configurar Recovery").click(setup_recovery, rec_pwd, rec_stat)

    with gr.Tab("🔄 Recuperar Vault"):
        rec_in  = gr.Textbox(label="Recovery Password", type="password")
        new_mp  = gr.Textbox(label="Nueva Contraseña Maestra", type="password")
        rec_out = gr.Textbox(label="Estado")
        gr.Button("Recuperar").click(recover_with_recovery, [rec_in, new_mp], rec_out)

    with gr.Tab("⚠️ Reset All"):
        master4 = gr.Textbox(label="Contraseña Maestra", type="password")
        res_all = gr.Textbox(label="Estado")
        gr.Button("Resetear todo").click(reset_all, master4, res_all)

    with gr.Tab("🔁 Cambiar Maestra"):
        old_mp = gr.Textbox(label="Contraseña Actual", type="password")
        new_mp2= gr.Textbox(label="Contraseña Nueva", type="password")
        ch_res = gr.Textbox(label="Estado")
        gr.Button("Cambiar master").click(reset_master_password, [old_mp, new_mp2], ch_res)

    with gr.Tab("📊 Verificar Fortaleza"):
        pwd_eval    = gr.Textbox(label="Contraseña a evaluar")
        str_out     = gr.Textbox(label="Nivel de Fortaleza")
        gr.Button("Verificar Fortaleza").click(evaluate_strength, pwd_eval, str_out)

    with gr.Tab("🚪 Salir"):
        exit_msg = gr.Textbox(label="Mensaje")
        gr.Button("Salir").click(lambda: exit_app(), None, exit_msg)

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://626f29a86e625d83de.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
